# Indexation hierarchique

L'indexation hierarchique est une fonctionnalité importante de pandas qui permet d'avoir plusieurs niveaux d'indexes sur un axe. 

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])  

In [4]:
data

a  1    0.443460
   2   -0.623263
   3   -1.012262
b  1   -1.352664
   3    0.165702
c  1   -0.256241
   2   -1.142411
d  2    1.774020
   3    1.639584
dtype: float64

In [ ]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [ ]:
data['b']

1   -0.769081
3    0.164928
dtype: float64

In [ ]:
data['b':'c']

b  1   -0.769081
   3    0.164928
c  1   -1.019170
   2    0.218949
dtype: float64

In [ ]:
data.loc[['b', 'd']]

b  1   -0.769081
   3    0.164928
d  2   -1.097378
   3   -0.079144
dtype: float64

On peut utiliser un niveau interne lors de la séléction:

In [ ]:
data.loc[:, 2]

a   -0.812517
c    0.218949
d   -1.097378
dtype: float64

In [ ]:
data.loc[:, 2:3]

a  2   -0.812517
   3    0.114533
b  3    0.164928
c  2    0.218949
d  2   -1.097378
   3   -0.079144
dtype: float64

L'indexation hierarchique joue un rôle primordial pour le changement du format de données et les opérations d'aggrégation comme le pivotage.

In [ ]:
data.unstack()

,1,2,3
a,-0.533840,-0.487353,0.143877
b,1.199166,NaN,-1.021224
c,-1.578583,0.710838,NaN
d,NaN,-0.767163,-0.900530


L'opération inverse est stack:

In [ ]:
data.unstack().stack()

a  1   -0.533840
   2   -0.487353
   3    0.143877
b  1    1.199166
   3   -1.021224
c  1   -1.578583
   2    0.710838
d  2   -0.767163
   3   -0.900530
dtype: float64

Pour un DataFrame, les deux axes peuvent avoir des indexes hierarchiques:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [ ]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [ ]:
frame.index.names = ['key1', 'key2']

In [ ]:
frame.columns.names = ['state', 'color']

In [ ]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

On peut créer également une MultiIndex à partir d'un tableau:

In [ ]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

**Tri des niveaux**

La fonction **swaplevel** permet d'échanger deux niveaux ou de trier les données par valeur selon un axe.

In [ ]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

La fonction **sort_index** permet de trier les données en utilisant les valeurs dans un seul niveau.

In [ ]:
frame.sort_index(level=1, axis=1)

state     Colorado  Ohio    
color        Green Green Red
key1 key2                   
a    1           2     0   1
     2           5     3   4
b    1           8     6   7
     2          11     9  10

In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

**Statistiques par niveau**

Plusieurs fonctions statistiques ont un argument pour spécifier le niveau qui peut être utilisé pour l'aggrégation.

In [ ]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [ ]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

# Combinaison et fusion des datasets

Les données contenues dans les objets pandas peuvent être combinées en utilisant:

* pandas.merge concatène ou empile plusieurs objets selon un axe
* La méthode combine_first permet de combiner les données chevauchantes afin de remplir les données manquantes dans un objet depuis l'autre objet


**Jointure des DataFrame**

Les opérations **merge** et **join** combinent les jeux de données "datasets" à travers des liaisons en utilisant une ou plusieurs clés de la même manière que les bases de données. La fonction merge de pandas est la fonction principale pour les jointures.

In [5]:
import pandas as pd

In [6]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b']})

In [7]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [8]:
df1

,key
0,b
1,b
2,a
3,c
4,a
5,a
6,b


In [9]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


Il s'agit d'un exemple de relation many-to-many: df1 a plusieurs enregistrements étiquetés a et b alors que df2 a uniquement un seul enregistrement pour chaque valeur dans la colonne key.

In [11]:
pd.merge(df1, df2)

,key,data2
0,b,1
1,b,1
2,b,1
3,a,0
4,a,0
5,a,0


Si aucune colonne n'est spécifiée, pandas utiliser les noms de colonnes en commun comme des clés, en pratique, il vaut mieux les spécifier explicitement.

In [12]:
pd.merge(df1, df2, on='key')

,key,data2
0,b,1
1,b,1
2,b,1
3,a,0
4,a,0
5,a,0


Si les colonnes sont différentes dans chaque objet, on peut les spécifier séparément:

In [14]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})

In [15]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

In [16]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Les valeurs 'c' et 'd' et les données associées sont manquantes dans le résultat. Par défaut, merge effectue une jointure interne "inner join", les clés dans le résultat représentent l'intersection ou les colonnes en commun des deux tables. Les autres options sont 'left', 'right' et 'outer'. Les jointures externes 'outer joins' prennent l'union des clés en combinant l'effet des jointures droites et gauches.

In [17]:
pd.merge(df1, df2, how='outer')

,key,data2
0,b,1.0
1,b,1.0
2,b,1.0
3,a,0.0
4,a,0.0
5,a,0.0
6,c,NaN
7,d,2.0


Option | Comportement
-- | --
'inner' | Utilise les combinaisons de clés retrouvées dans les deux tables
'left' | Utilise toutes les combinaisons de clés retrouvées dans la table gauche
'right' | Utilise toutes les combinaisons de clés retrouvées dans la table droite
'outer' | Utilise toutes les combinaisons de clés retrouvées dans les deux tables

Les jointures many-to-many produisent le produit cartésien des enregistrements.

In [18]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

In [19]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

In [20]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [21]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [22]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [23]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


On peut joindre avec plusieurs clés en passant les noms de colonnes.

In [24]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

In [26]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [27]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


La fonction merge accepte l'argument **suffixes** afin de spécifier les chaînes de caractères à ajouter à la fin des colonnes en commun.

In [28]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [29]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


Argument | Description
-- | --
left | Jointure gauche
right | Jointure droite
how | 'inner', 'outer', 'left' ou 'right', par défaut 'inner'
on | Le nom de colonnes à utiliser pour la jointure
left_on | Les colonnes du DataFrame gauche à utiliser comme clés de jointure
right_on | Les dolonnes du DataFrame droit à utiliser comme clés de jointure
left_index | Utiliser l'indexe du DataFrame gauche comme clé de jointure
right_index | Utiliser l'indexe du DataFrame droit comme clé de jointure
sort | Trier les données fusionnées d'une manière lexicographique, par défaut 'True'
suffixes | Tuple de valeurs strings à ajouter en cas de chevauchement, par défaut ('_x', '_y')
copy | Si False, évite de copier les données dans la structure de données du résultat
indicator | Ajoute une colonne spéciale _merge pour indiquer la source de chaque enregistrement, les valeurs possibles sont: 'left_only', 'right_only' ou 'both' en se basant sur l'origine de chaque enregistrement.

**Les fusions à partir de l'index**

Dans certains cas, les clés de fusion dans un DataFrame sont trouvés dans son indexe, dans ce cas, on peut passer left_index=True ou right_index=True ou les deux pour indiquer que l'indexe doit être utilisée comme clé de fusion.

In [30]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})

In [31]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [32]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [33]:
right1

,group_val
a,3.5
b,7.0


In [34]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [35]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Pour les données avec des indexes hiérarchiques:

In [36]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

In [37]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [38]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [39]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

Dans ce cas, il faut indiquer plusieurs colonnes pour fusionner en utilisant une liste:

In [40]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [41]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


On peut également utiliser les indexes des deux côtés de la fusion:

In [42]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

In [44]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [45]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [46]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [47]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


Le DataFrame a une méthode **join** pour fusionner par indexe, elle peut être aussi utilisée pour combiner plusieurs DataFrame ayant les mêmes ou des indexes similaires mais des colonnes différentes.

In [48]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


On peut passer une liste de DataFrames à joindre comme alternative de la fonction **concat**:

In [49]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [50]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [51]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [52]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


**Concaténation selon un axe**

Un autre type de fusion est la concaténation ou l'empilation, elle peut être effectuée en utilisant la fonction **concat**.

In [53]:
s1 = pd.Series([0, 1], index=['a', 'b'])

In [55]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [56]:
s3 = pd.Series([5, 6], index=['f', 'g'])

In [57]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

Par défaut, **concat** utilise axis=0 produisant un autre objet Series, si axis=1, le résultat sera un DataFrame.

In [58]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


Si aucun chevauchement n'existe sur l'autre axe, on peut obtenir l'intersection en passant join='inner'.

In [59]:
s4 = pd.concat([s1, s3])

In [60]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [61]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [62]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [64]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [65]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [66]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


Dans le cas de combinaison de plusieurs Series selon l'axe axis=1, les clés deviennent les entêtes de colonnes:

In [67]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


Et la même logique pour les DataFrames:

In [68]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                                        columns=['one', 'two'])

In [69]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [70]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [71]:
df2

,three,four
a,5,6
c,7,8


In [72]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Si on passe un dictionnaire au lieu d'une liste, les clés du dictionnaire seront utilisées pour l'option keys:

In [73]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

On peut nommer les axes crées avec l'argument names:

In [74]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

On peut passer ignore_index=True pour le cas où l'index des lignes ne contiennent pas de données significatives:

In [75]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [76]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [77]:
df1

,a,b,c,d
0,0.529858,-0.617711,1.010303,-0.056326
1,0.947731,-0.423241,-0.815774,0.843157
2,0.009577,-0.880614,-1.270547,2.144702


In [78]:
df2

,b,d,a
0,0.158106,-1.482532,1.111667
1,-1.767299,-0.833714,-0.139240


In [79]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.529858,-0.617711,1.010303,-0.056326
1,0.947731,-0.423241,-0.815774,0.843157
2,0.009577,-0.880614,-1.270547,2.144702
3,1.111667,0.158106,NaN,-1.482532
4,-0.139240,-1.767299,NaN,-0.833714


Argument | Description
-- | --
objs | Liste ou dictionnaire des objets à concaténer
axis | L'axe à utiliser lors de la concaténation
join | 'inner' pour l'intersection ou 'outer' pour l'union
join_axes | Spécifier les indexes à utiliser pour les autres axes
keys | Valeurs à associer avec les objets à concaténer
levels | Spécifier les indexes à utiliser comme indexe hiérarchique
names | Les noms pour les indexes hiérarchiques crées
ignore_index | Ne pas préserver les indexes selon les axes de concaténation.

**Combinaison des données qui se chevauchent**

In [81]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [82]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [83]:
b[-1] = np.nan

In [84]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [85]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [86]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [87]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

**combine_first** permet de patcher les valeurs manquantes depuis l'objet passé en paramètre:

In [88]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

In [89]:
df2 = pd.DataFrame({'a': [5, 4, np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [90]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [91]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [92]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# Pivotage

**Reformatage à l'aide de l'indexation hiérarchique**

L'indexation hiérarchique permet de réarranger les données dans un DataFrame, il existe deux actions primaires:

* stack: effectue la rotation des colonnes vers les lignes
* unstack: effectue l'opération inverse

In [93]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                                     name='number'))

In [94]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


L'utilisation de la méthode **stack** permet de pivoter les colonnes en des lignes produisant une Serie:

In [95]:
result = data.stack()

In [96]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

**unstack** peut être utilisée pour revenir vers une Serie avec une indexe hiérarchique:

In [97]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Par défaut, le niveau le plus interne est non empilé, on peut désempiler un niveau spécifique en passant le numéro du niveau ou son nom:

In [98]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [99]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


La désempilation peut produire des valeurs manquantes:

In [101]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [102]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [103]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [104]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [105]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Quand on désempile, le niveau disempilé devient le niveau le plus bas:

In [106]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))

In [107]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [108]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10